
# Use scikit-learn and AI lifecycle capabilities to predict Boston house prices with `ibm-watson-machine-learning`

This notebook contains steps and code to demonstrate support of AI Lifecycle features in Watson Machine Learning Service. It contains steps and code to work with [ibm-watson-machine-learning](https://pypi.python.org/pypi/ibm-watson-machine-learning) library available in PyPI repository. It also introduces commands for getting model and training data, persisting model, deploying model, scoring it, updating the model and redeploying it.

Some familiarity with Python is helpful. This notebook uses Python 3.

## Learning goals

The learning goals of this notebook are:

-  Download an externally trained scikit-learn model with dataset.
-  Persist an external model in Watson Machine Learning repository.
-  Deploy model for online scoring using client library.
-  Score sample records using client library.
-  Update previously persisted model.
-  Redeploy model in-place.
-  Scale deployment.


## Contents

This notebook contains the following parts:

1. [Setup](#setup)
2. [Download externally created scikit model and data](#download)
3. [Persist externally created scikit model](#persistence)
4. [Deploy and score in a Cloud](#scoring)
5. [Persist new version of the model](#update_model)
6. [Redeploy new version of the model](#redeploy)
7. [Deployment scaling](#scale)
8. [Clean up](#cleanup)
9. [Summary and next steps](#summary)

<a id="setup"></a>
## 1. Set up the environment

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Create a <a href="https://cloud.ibm.com/catalog/services/watson-machine-learning" target="_blank" rel="noopener no referrer">Watson Machine Learning (WML) Service</a> instance (a free plan is offered and information about how to create the instance can be found <a href="https://dataplatform.cloud.ibm.com/docs/content/wsj/admin/create-services.html?context=cpdaas&audience=wdp" target="_blank" rel="noopener no referrer">here</a>).

### Connection to WML

Authenticate the Watson Machine Learning service on IBM Cloud. You need to provide platform `api_key` and instance `location`.

You can use [IBM Cloud CLI](https://cloud.ibm.com/docs/cli/index.html) to retrieve platform API Key and instance location.

API Key can be generated in the following way:
```
ibmcloud login
ibmcloud iam api-key-create API_KEY_NAME
```

In result, get the value of `api_key` from the output.


Location of your WML instance can be retrieved in the following way:
```
ibmcloud login --apikey API_KEY -a https://cloud.ibm.com
ibmcloud resource service-instance WML_INSTANCE_NAME
```

In result, get the value of `location` from the output.

**Tip**: Your `Cloud API key` can be generated by going to the [**Users** section of the Cloud console](https://cloud.ibm.com/iam#/users). From that page, click your name, scroll down to the **API Keys** section, and click **Create an IBM Cloud API key**. Give your key a name and click **Create**, then copy the created key and paste it below. You can also get a service specific url by going to the [**Endpoint URLs** section of the Watson Machine Learning docs](https://cloud.ibm.com/apidocs/machine-learning).  You can check your instance location in your  <a href="https://cloud.ibm.com/catalog/services/watson-machine-learning" target="_blank" rel="noopener no referrer">Watson Machine Learning (WML) Service</a> instance details.

You can also get service specific apikey by going to the [**Service IDs** section of the Cloud Console](https://cloud.ibm.com/iam/serviceids).  From that page, click **Create**, then copy the created key and paste it below.

**Action**: Enter your `api_key` and `location` in the following cell.

In [ ]:
api_key = 'PASTE YOUR PLATFORM API KEY HERE'
location = 'PASTE YOUR INSTANCE LOCATION HERE'

In [ ]:
wml_credentials = {
    "apikey": api_key,
    "url": 'https://' + location + '.ml.cloud.ibm.com'
}

### Install and import the `ibm-watson-machine-learning` package
**Note:** `ibm-watson-machine-learning` documentation can be found <a href="http://ibm-wml-api-pyclient.mybluemix.net/" target="_blank" rel="noopener no referrer">here</a>.

In [ ]:
!pip install -U ibm_watson_machine_learning

In [ ]:
from ibm_watson_machine_learning import APIClient

client = APIClient(wml_credentials)

### Working with spaces

First of all, you need to create a space that will be used for your work. If you do not have space already created, you can use [Deployment Spaces Dashboard](https://dataplatform.cloud.ibm.com/ml-runtime/spaces) to create one.

- Click New Deployment Space
- Create an empty space
- Select Cloud Object Storage
- Select Watson Machine Learning instance and press Create
- Copy `space_id` and paste it below

**Tip**: You can also use SDK to prepare the space for your work. More information can be found [here](https://github.com/IBM/watson-machine-learning-samples/blob/master/cloud/notebooks/python_sdk/instance-management/Space%20management.ipynb).

**Action**: Assign space ID below

In [ ]:
space_id = 'PASTE YOUR SPACE ID HERE'

You can use `list` method to print all existing spaces.

In [ ]:
client.spaces.list(limit=10)

To be able to interact with all resources available in Watson Machine Learning, you need to set **space** which you will be using.

In [ ]:
client.set.default_space(space_id)

<a id="download"></a>
## 2. Download an externally created scikit model and data
In this section, you will download externally created scikit models and data used for training it.

In [ ]:
! pip install wget

In [ ]:
import os
import wget

data_dir = 'BOSTON_HOUSE_PRICES_DATA'
if not os.path.isdir(data_dir):
    os.mkdir(data_dir)

model_path = os.path.join(data_dir, 'boston_house_prices_model.tar.gz')
new_model_path = os.path.join(data_dir, 'new_boston_house_prices_model.tar.gz')
if not os.path.isfile(model_path):
    wget.download("https://github.com/IBM/watson-machine-learning-samples/raw/master/cloud/models/scikit/boston_house_price/model/boston_house_prices_model.tar.gz", out=data_dir)  
if not os.path.isfile(new_model_path):
    wget.download("https://github.com/IBM/watson-machine-learning-samples/raw/master/cloud/models/scikit/boston_house_price/model/new_boston_house_prices_model.tar.gz", out=data_dir)  

In [ ]:
from sklearn import datasets
import pandas as pd

boston_data = datasets.load_boston()
boston_df = pd.DataFrame(boston_data.data)
boston_df.columns = boston_data.feature_names
boston_df['PRICE'] = boston_data.target

In [ ]:
train_df = boston_df
test_df = boston_df.drop(['PRICE'], axis=1)

<a id="persistence"></a>
## 3. Persist the externally created scikit model

In this section, you will learn how to store your model in Watson Machine Learning repository by using the Watson Machine Learning Client.

### 3.1: Publish model

#### Publish model in the Watson Machine Learning repository on Cloud.

Define model name, autor name and email.

In [ ]:
sofware_spec_uid = client.software_specifications.get_id_by_name("runtime-23.1-py3.10")

In [ ]:
metadata = {
            client.repository.ModelMetaNames.NAME: 'External scikit model',
            client.repository.ModelMetaNames.TYPE: 'scikit-learn_1.1',
            client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: sofware_spec_uid
}

published_model = client.repository.store_model(
    model=model_path,
    meta_props=metadata,
    training_data=train_df)

### 3.2: Get model details

In [ ]:
import json

published_model_uid = client.repository.get_model_id(published_model)
model_details = client.repository.get_details(published_model_uid)
print(json.dumps(model_details, indent=2))

### 3.3 Get all models

In [ ]:
models_details = client.repository.list_models(limit=10)

<a id="scoring"></a>
## 4. Deploy and score in a Cloud

In this section you will learn how to create online scoring and to score a new data record by using the Watson Machine Learning Client.

### 4.1: Create model deployment

#### Create an online deployment for the published model

In [ ]:
metadata = {
    client.deployments.ConfigurationMetaNames.NAME: "Deployment of external scikit model",
    client.deployments.ConfigurationMetaNames.ONLINE: {}
}

created_deployment = client.deployments.create(published_model_uid, meta_props=metadata)

**Note**: Here we use deployment url saved in published_model object. In next section, we show how to retrive deployment url from Watson Machine Learning instance.

In [ ]:
deployment_uid = client.deployments.get_id(created_deployment)

Now you can print an online scoring endpoint. 

In [ ]:
scoring_endpoint = client.deployments.get_scoring_href(created_deployment)
print(scoring_endpoint)

You can also list existing deployments.

In [ ]:
client.deployments.list(limit=10)

### 4.2: Get deployment details

In [ ]:
print(json.dumps(client.deployments.get_details(deployment_uid), indent=2))

### 4.3: Score

You can use below method to do test scoring request against deployed model.

**Action**: Prepare scoring payload with records to score.

In [ ]:
score_0 = list(test_df.iloc[0])
score_1 = list(test_df.iloc[1])

In [ ]:
scoring_payload = {"input_data": [{"values": [score_0, score_1]}]}

Use ``client.deployments.score()`` method to run scoring.

In [ ]:
predictions = client.deployments.score(deployment_uid, scoring_payload)

In [ ]:
print(json.dumps(predictions, indent=2))

<a id="update_model"></a>
## 5. Persist new version of the model

In this section, you'll learn how to store new version of your model in Watson Machine Learning repository by using the Watson Machine Learning Client.

### 5.1: Publish new version of the model

Save the current model version. 

In [ ]:
print(json.dumps(client.repository.create_model_revision(published_model_uid), indent=2))

Define new model name and update model content. 

In [ ]:
metadata = {
            client.repository.ModelMetaNames.NAME: 'External scikit model - updated'
}

published_model = client.repository.update_model(
    model_uid=published_model_uid,
    update_model=new_model_path,
    updated_meta_props=metadata
)

Save new model revision of the updated model. 

In [ ]:
new_model_revision = client.repository.create_model_revision(published_model_uid)
print(json.dumps(new_model_revision, indent=2))

**Note:** Model revisions can be identified by model `id` and `rev` number. 

Get model `rev` number from creation details:

In [ ]:
rev_id = new_model_revision['metadata'].get('rev')

You can list existing revisions of the model. 

In [ ]:
client.repository.list_models_revisions(published_model_uid)

### 5.2: Get model details

In [ ]:
import json

published_model_uid = client.repository.get_model_id(published_model)
model_details = client.repository.get_details(published_model_uid)

print(json.dumps(model_details, indent=2))

<a id="redeploy"></a>
## 6. Redeploy new version of the model

In this section, you'll learn how to redeploy new version of the model by using the Watson Machine Learning Client.

### 6.1 Redeploy model

In [ ]:
metadata = {
    client.deployments.ConfigurationMetaNames.ASSET: {
        "id": published_model_uid,
        "rev": rev_id
    }
}
    
updated_deployment = client.deployments.update(deployment_uid=deployment_uid, changes=metadata)

Wait for the deployment update: 

In [ ]:
import time

status = None
while status not in ['ready', 'failed']:
    print('.', end=' ')
    time.sleep(2)
    deployment_details = client.deployments.get_details(deployment_uid)
    status = deployment_details['entity']['status'].get('state')
    
print("\nDeployment update finished with status: ", status)

### 6.2 Get updated deployment details

In [ ]:
print(json.dumps(client.deployments.get_details(deployment_uid), indent=2))

<a id="scale"></a>
## 7. Deployment scaling

In this section, you'll learn how to scale your deployment by creating more copies of stored model with Watson Machine Learning Client.  
This feature is for providing High-Availability and to support higher throughput

### 7.1 Scale deployment

In this example, 2 deployment copies will be made.

In [ ]:
metadata = {
    client.deployments.ConfigurationMetaNames.NAME: "Deployment of external scikit model - scaling",
    client.deployments.ConfigurationMetaNames.HARDWARE_SPEC: {
        "name": "S",
        "num_nodes": 2
    }
}

In [ ]:
scaled_deployment = client.deployments.update(deployment_uid, metadata)

### 7.2 Get scaled deployment details

In [ ]:
print(json.dumps(client.deployments.get_details(deployment_uid), indent=2))

### 7.3 Score updated deployment

You can use below method to do test scoring request against deployed model.

**Action**: Prepare scoring payload with records to score.

In [ ]:
score_0 = list(test_df.iloc[0])
score_1 = list(test_df.iloc[1])

In [ ]:
scoring_payload = {"input_data": [{"values": [score_0, score_1]}]}

Use client.deployments.score() method to run scoring.

In [ ]:
predictions = client.deployments.score(deployment_uid, scoring_payload)

In [ ]:
print(json.dumps(predictions, indent=2))

<a id="cleanup"></a>
## 8. Clean up

If you want to clean up all created assets:
- experiments
- trainings
- pipelines
- model definitions
- models
- functions
- deployments

please follow up this sample [notebook](https://github.com/IBM/watson-machine-learning-samples/blob/master/cloud/notebooks/python_sdk/instance-management/Machine%20Learning%20artifacts%20management.ipynb).

<a id="summary"></a>
## 9. Summary and next steps

 You successfully completed this notebook! You learned how to use scikit-learn machine learning as well as Watson Machine Learning for model creation and deployment. Check out our _[Online Documentation](https://dataplatform.cloud.ibm.com/docs/content/wsj/getting-started/welcome-main.html)_ for more samples, tutorials, documentation, how-tos, and blog posts. 

### Authors

**Daniel Ryszka**, Software Engineer  
**Jan Sołtysik**, Intern 

Copyright © 2020 IBM. This notebook and its source code are released under the terms of the MIT License.